# Playground for strokes graph

In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt
import momepy
import networkx as nx
import folium
from itertools import combinations, product
import shapely
import numpy as np
import pickle

In [3]:
streets = gpd.read_file(momepy.datasets.get_path("bubenec"), layer="streets")

In [4]:
# Clean data
streets = momepy.remove_false_nodes(streets)
# Transform into primal graph
G_primal = momepy.gdf_to_nx(streets, approach="primal", preserve_index=True)
points_primal, lines_primal = momepy.nx_to_gdf(G_primal, points=True, lines=True)
# Use COINS on primal graph edges
coins = momepy.COINS(lines_primal)
# List the stroke for each edge
stroke_attribute = coins.stroke_attribute()
# List each edge for each stroke
stroke_gdf = coins.stroke_gdf()
stroke_gdf["edge_ids"] = [stroke_attribute[stroke_attribute == stroke_id].index.values for stroke_id in stroke_gdf.index.values]
# Add stroke ID to each edge
nx.set_edge_attributes(G_primal, {e: int(stroke_attribute[G_primal.edges[e]["index_position"]]) for e in G_primal.edges}, "stroke_id")
points_primal, lines_primal = momepy.nx_to_gdf(G_primal, points=True, lines=True)

In [5]:
# stroke_gdf_viz = stroke_gdf.copy()
# stroke_gdf_viz["id"] = stroke_gdf_viz.index
# lines_primal_viz = lines_primal.copy()
# lines_primal_viz["edge_id"] = lines_primal.index
# m = stroke_gdf_viz.explore(tiles="cartodb-positron", column="id", cmap="Set2", name="strokes")
# lines_primal_viz.explore(m=m, column="stroke_id", name = "lines", cmap="Set2")
# folium.LayerControl().add_to(m)
# m

In [6]:
stroke_gdf.loc[stroke_gdf.index.values[0]]

n_segments                                                    8
geometry      LINESTRING (1603278.8993584276 6463669.1855955...
edge_ids                                         [0, 3, 15, 27]
Name: 0, dtype: object

In [64]:
def find_geom(linestring, point):
    if point == linestring.coords[0]:
        geom = [np.array(val) for val in linestring.coords[:2]]
    else:
        geom = [np.array(val) for val in linestring.coords[-2:]]
    return np.array(geom[0] - geom[1])

In [65]:
def angle(a, b):
    angle = np.rad2deg(np.arccos(np.dot(a, b)/(np.linalg.norm(a) * np.linalg.norm(b))))
    if angle > 90:
        angle = 180 - angle
    return angle

In [66]:
# Create stroke graph
G_stroke = nx.Graph()
G_stroke.graph["crs"] = G_primal.graph["crs"]
# Create a node for each stroke with the right features
G_stroke.add_nodes_from([[int(idx), {(attr if attr != "geometry" else "geometry_stroke"):stroke_gdf.loc[idx][attr] for attr in list(stroke_gdf)}] for idx in stroke_gdf.index.values])
# For all node, put its geometry at the center of the LineString
for n in G_stroke.nodes:
    G_stroke.nodes[n]["geometry"] = stroke_gdf.iloc[n].geometry.interpolate(0.5, normalized=True)
    G_stroke.nodes[n]["x"] = G_stroke.nodes[n]["geometry"].xy[0]
    G_stroke.nodes[n]["y"] = G_stroke.nodes[n]["geometry"].xy[1]
    G_stroke.nodes[n]["length"] = G_stroke.nodes[n]["geometry_stroke"].length
# Find strokes intersecting
for n in G_primal.nodes:
    strokes_present = [G_primal.edges[e]["stroke_id"] for e in G_primal.edges(n, keys=True)]
    # If strokes intersecting, add the edge if not already present
    if len(set(strokes_present)) > 1:
        for u, v in combinations(set(strokes_present), 2):
            # Find all edges touching the node for both strokes checked
            edges_u = [e for e in G_primal.edges(n, keys=True) if G_primal.edges[e]["stroke_id"] == u]
            edges_v = [e for e in G_primal.edges(n, keys=True) if G_primal.edges[e]["stroke_id"] == v]
            angle_list = []
            angle_dict = {}
            # Choose the smallest list as number of angles kept
            chosen, other = sorted([edges_u, edges_v], key=len)
            # Find the angles
            for ce, oe in list(product(chosen, other)):
                point = [G_primal.nodes[n]["x"], G_primal.nodes[n]["y"]]
                gc = find_geom(G_primal.edges[ce]["geometry"], point)
                go = find_geom(G_primal.edges[oe]["geometry"], point)
                if ce in angle_dict:
                    angle_dict[ce].append(angle(gc, go))
                else:
                    angle_dict[ce]= [angle(gc, go)]
            # Keep the smallest angles
            angle_list = [min(angle_dict[ekey]) for ekey in angle_dict]
            # TODO solve angles
            if G_stroke.has_edge(u, v):
                G_stroke.edges[u, v]["angles"] += angle_list
                G_stroke.edges[u, v]["number_connections"] = len(G_stroke.edges[u, v]["angles"])
            else:
                G_stroke.add_edge(u, v, geometry = shapely.LineString([G_stroke.nodes[u]["geometry"], G_stroke.nodes[v]["geometry"]]), number_connections=len(angle_list), angles=angle_list)

In [ ]:
nx.set_node_attributes(G_stroke, nx.betweenness_centrality(G_stroke), "stroke_betweenness")
nx.set_node_attributes(G_stroke, nx.closeness_centrality(G_stroke), "stroke_closeness")
nx.set_node_attributes(G_stroke, dict(nx.degree(G_stroke)), "stroke_degree")
for n in G_stroke.nodes:
    G_stroke.nodes[n]["stroke_connectivity"] = sum([G_stroke.edges[e]["number_connections"] for e in G_stroke.edges(n)])
    G_stroke.nodes[n]["stroke_access"] = G_stroke.nodes[n]["stroke_connectivity"] - G_stroke.nodes[n]["stroke_degree"]
    angles = [val for e in G_stroke.edges(n) if G_stroke.edges[e]["angles"] for val in G_stroke.edges[e]["angles"]]
    G_stroke.nodes[n]["stroke_orthogonality"] = sum(angles) / G_stroke.nodes[n]["stroke_connectivity"]
    G_stroke.nodes[n]["stroke_spacing"] = G_stroke.nodes[n]["length"] / G_stroke.nodes[n]["stroke_connectivity"]

Save as pickle

In [ ]:
# after iteration finished, save results:
with open('stroke_graph_clse.pickle', 'wb') as handle:
    pickle.dump(G_stroke, handle, protocol=pickle.HIGHEST_PROTOCOL)

**Visualize final results**

In [68]:
stroke_gdf_viz = stroke_gdf.copy()
stroke_gdf_viz["id"] = stroke_gdf_viz.index
lines_primal_viz = lines_primal.copy()
lines_primal_viz["edge_id"] = lines_primal.index
points_stroke, lines_stroke = momepy.nx_to_gdf(G_stroke, points=True, lines=True)
m = stroke_gdf_viz.explore(tiles="cartodb-positron", column="id", cmap="tab10", name="strokes")
points_primal.explore(m=m, name="points_primal")
lines_primal_viz.explore(m=m, name="lines_primal")
points_stroke.explore(m=m, color="black", marker_kwds={"radius":10}, name="points_stroke")
lines_stroke.explore(m=m, color="blue", name="lines_stroke")
folium.LayerControl().add_to(m)
m

/var/folders/mb/_ysy1pzs13qgnh9b942_7lkh0000gn/T/ipykernel_54955/2550495861.py:5: UserWarning: Approach is not set. Defaulting to 'primal'.
  points_stroke, lines_stroke = momepy.nx_to_gdf(G_stroke, points=True, lines=True)
